In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

DATAFILE = "data/Asynchron_data_in.dat"
df = pd.read_csv(DATAFILE, sep=" ")
t = df["time"].values
y = df["position"].values
a = df["acceleration"].values
# u = np.zeros((y.shape[0], 1))
Ts = t[1] - t[0]

print(df)


       time   position  acceleration
0     0.011   0.938599           NaN
1     0.060        NaN      0.019471
2     0.150        NaN     -0.016152
3     0.260        NaN      0.021454
4     0.360        NaN      0.004004
..      ...        ...           ...
235  19.571  38.916309           NaN
236  19.640        NaN      0.018039
237  19.760        NaN      0.034991
238  19.830        NaN     -0.005509
239  19.960        NaN      0.014985

[240 rows x 3 columns]


In [51]:
# Define system matrices // Gamma filter
Ad = np.array([[1, Ts, 0.5 * Ts**2], [0, 1, Ts], [0, 0, 1]])
Bd = np.array([[0], [0], [0]])
C = np.array([[1, 0, 0]])  # dont think this is correct
# D = np.array([0])
Gd = np.array([[0.5 * Ts**2], [Ts], [1]])

# Initialization
x_dach = np.array([y[0], 0])
P_dach = 50 * np.eye(2)

print(Ad.shape)


(3, 3)


In [52]:
y_rand_temp = []
y_rand_var = []

for i in y:
    if str(i) != "nan":
        # print(i)
        y_rand_temp.append(i)

# print(len(y_rand_temp))

for i in range(0, 3, 1):
    y_rand_temp2 = random.choices(y_rand_temp[0 : len(y_rand_temp)], k=10)

    # print(y_rand_temp)

    y_rand_var.append(np.var(y_rand_temp))
    # y_rand_mean = np.mean(y_rand_temp)
    # y_rand_var_temp = np.var(y_rand_temp)
    # y_rand_var = y_rand_var_temp if y_rand_var_temp < y_rand_var else y_rand_var
    # if (y_rand_var_temp < y_rand_var):
    #     y_rand_var = y_rand_var_temp

# print(y_rand_temp2)

R = np.eye(3) * min(y_rand_var)
print(R)
Q = np.eye(3) * 1e-4

# print(y)  # todo


[[212.79783295   0.           0.        ]
 [  0.         212.79783295   0.        ]
 [  0.           0.         212.79783295]]


In [66]:
# Kalman filter
s = np.zeros(len(y))
v = np.zeros(len(y))
K1 = np.zeros(len(y))
K2 = np.zeros(len(y))
P_tilde1 = np.zeros(len(y))
P_tilde2 = np.zeros(len(y))
P_tilde3 = np.zeros(len(y))
P_tilde4 = np.zeros(len(y))
d_y = np.zeros(len(y))

x_dach = np.zeros(len(y))
P_dach = np.zeros(len(y))
K = np.zeros(len(y))

P_dach[0] = 100 * Q[0][0]
x_dach[0] = y[0]

print(K.shape)
print(P_dach[3] / (P_dach[3] + R[0][0]))

for k in range(len(y) - 1):
    # print(d_y[k])
    # x_dach =
    K[k] = P_dach[k] / (P_dach[k] + R[0][0])
    x_dach[k + 1] = x_dach[k] + K[k] * (y[k] - x_dach[k])
    d_y[k] = y[k]


(240,)
0.0
